In [8]:
import pandas as pd
import psycopg2
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from math import radians

USER_ID = 461  # Change this to whatever user
PATH_DATA = '../../data/user_sequences/'
EXTENSION_TEXT = '.txt'
PATH_USER_DATA = PATH_DATA + f'user_{USER_ID}_sequence' + EXTENSION_TEXT
DISTANCE_THRESHOLD = 200  # In meters

# Database connection configurations
POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'leopoldo'
POSTGRES_PASSWORD = '1234'
POSTGRES_DBNAME = 'suscity'

db_uri = f'postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}'

conn = psycopg2.connect(database=POSTGRES_DBNAME, user=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD, host=POSTGRES_ADDRESS,
                        port=POSTGRES_PORT)

conn.autocommit = True

file = open(PATH_USER_DATA, 'w')

session_file_name = 'user_sessions_number.txt'

In [9]:
sessions = pd.read_sql_query(f'SELECT * FROM session WHERE user_id = {USER_ID} ORDER BY start_time', conn)

# Unique sessions this user has
user_sessions = list(sessions['session_id'])

max_sessions = 5
counter = 0

prev_processed_session = None

for index, session_id in enumerate(user_sessions):
    processed_session = pd.read_sql_query(f'SELECT * FROM processed_session WHERE session_id = {session_id}', conn)

    if processed_session.empty:
        continue

    session_start_lat = processed_session['lat_start'].values[0]
    session_start_lon = processed_session['lon_start'].values[0]
    session_end_lat = processed_session['lat_end'].values[0]
    session_end_lon = processed_session['lon_end'].values[0]
    session_start_time = processed_session['seconds_start'].values[0]
    date_time = pd.to_datetime(session_start_time, unit='s', origin='unix')
    date_str = date_time.strftime('%d/%m/%Y, %H:%M:%S')
    week_index = date_time.weekday()

    # Codigo horrivel eu sei

    if week_index > 4:  # Check if session happened on a week day
        continue

    if session_start_lat == session_end_lat and session_start_lon == session_end_lon:  # Filter some sessions that have same start and end point
        continue

    if prev_processed_session is None:  # If this is the first session, write it to the file
        prev_processed_session = processed_session
        file.write(f'{week_index},{session_start_lat},{session_start_lon}\n')
        file.write(f'{week_index},{session_end_lat},{session_end_lon}\n')
        continue

    prev_session_end_lat = prev_processed_session['lat_end'].values[0]
    prev_session_end_lon = prev_processed_session['lon_end'].values[0]
    prev_session_start_time = prev_processed_session['seconds_start'].values[0]
    prev_date_time = pd.to_datetime(prev_session_start_time, unit='s', origin='unix')
    prev_week_index = prev_date_time.weekday()


    #Calculate distance between previous session end and current session start
    prev_end = np.array([[radians(prev_session_end_lat), radians(prev_session_end_lon)]])
    curr_start = np.array([[radians(session_start_lat), radians(session_start_lon)]])
    dist = haversine_distances(prev_end, curr_start)[0,0] * 6371000 #Multiply by radius of the earth in meters, to get the distance in meters

    if dist > DISTANCE_THRESHOLD:
        #If the distnace is above the threshold, create a "fictionary" session that join the end of the previous one, to the start of the current one
        file.write(f'{week_index},{session_start_lat},{session_start_lon}\n')
        file.write(f'{week_index},{session_end_lat},{session_end_lon}\n')
    else:
        # If the distance is below the threshold, "connect" end of last session to end of the current one
        file.write(f'{week_index},{session_end_lat},{session_end_lon}\n')

    prev_processed_session = processed_session

file.close()


C:\Users\alexl\AppData\Local\Temp\ipykernel_9848\3142660629.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sessions = pd.read_sql_query(f'SELECT * FROM session WHERE user_id = {USER_ID} ORDER BY start_time', conn)
C:\Users\alexl\AppData\Local\Temp\ipykernel_9848\3142660629.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  processed_session = pd.read_sql_query(f'SELECT * FROM processed_session WHERE session_id = {session_id}', conn)
C:\Users\alexl\AppData\Local\Temp\ipykernel_9848\3142660629.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please c